# 해외 여행, 언제 갈래? [여행정보 크롤링]

# PART 1 환율

In [6]:
import requests

import telegram

# (1) 환율정보 함수 만들기 [def 함수]

In [32]:
from bs4 import BeautifulSoup

In [73]:
def 환율(country =''):
    URL = 'https://finance.naver.com/marketindex/exchangeList.naver'
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.select('tbody > tr')
    final_result = {}
    result1 = soup.select('.tit')
    result2 = soup.select('.sale')

    for result in results:
        #tbody > tr 검색한 것이 통화량 환율 등 한 행이니까 반복문을 돌리고 있음
        #한 행별로 tit sale을 검색한다.(select_one 사용)
        title = result.select_one('.tit').text.strip()
        sale = result.select_one('.sale').text.strip()

        temp = {
            '통화명': title.split(),
            '환율': sale
        }

        final_result.setdefault(title.split()[0], sale) #딕셔너리로 만들어라!!! 키는 타이틀 스플릿해서 앞에것만 가져오라, 밸류는 sale 넣어라 (split)
        # {미국 : 1393.00, 유럽연합 : 1389.87} #정리한 다음에 미국, 유럽연합으로 함수의 인자(파라미터)를 받아라
    if country == '':
        return final_result
    else:
        return final_result[country]

     #실습자료 보기

    #함수 안에 있는 변수는 밖에서 못씀
    #지역변수 전역변수
    #리턴을 달아주는 이유는 함수 안에서 실행돼서 밖에서 못쓰니까
    #어떻게든 전달위해!!
    #문자열로 만드는 연습을 해라

##(시현)내가 가장 가고 싶은 나라 2개 추출해보기 (미국 & 유럽연합)

In [34]:
환율()
#딕셔너리에 저장

{'미국': '1,391.30',
 '유럽연합': '1,385.46',
 '일본': '968.70',
 '중국': '197.89',
 '홍콩': '177.25',
 '대만': '44.35',
 '영국': '1,582.46',
 '오만': '3,613.67',
 '캐나다': '1,040.89',
 '스위스': '1,443.03',
 '스웨덴': '127.53',
 '호주': '931.61',
 '뉴질랜드': '820.80',
 '체코': '56.26',
 '칠레': '1.49',
 '터키': '75.94',
 '몽골': '0.43',
 '이스라엘': '403.44',
 '덴마크': '186.30',
 '노르웨이': '134.41',
 '사우디아라비아': '369.88',
 '쿠웨이트': '4,503.32',
 '바레인': '3,690.26',
 '아랍에미리트': '378.78',
 '요르단': '1,962.34',
 '이집트': '71.57',
 '태국': '37.57',
 '싱가포르': '986.35',
 '말레이시아': '304.64',
 '인도네시아': '9.31',
 '카타르': '379.74',
 '카자흐스탄': '2.92',
 '브루나이': '986.35',
 '인도': '17.44',
 '파키스탄': '5.82',
 '방글라데시': '13.43',
 '필리핀': '24.16',
 '멕시코': '69.59',
 '브라질': '270.52',
 '베트남': '5.88',
 '남아프리카': '78.62',
 '러시아': '22.66',
 '헝가리': '3.45',
 '폴란드': '292.66',
 '스리랑카': '3.83',
 '알제리': '9.88',
 '케냐': '11.54',
 '콜롬비아': '0.31',
 '탄자니아': '0.60',
 '네팔': '10.90',
 '루마니아': '280.26',
 '리비아': '280.68',
 '마카오': '172.00',
 '미얀마': '0.66',
 '에티오피아': '26.34',
 '우즈베키스탄': '0.1

In [35]:
환율('유럽연합')
#환율 미국 검색시 그래프랑 & 특정 환율만 보이게!


'1,385.46'

# (2) 3개월간 국제 시장 환율 변동 측정 & 텔레그램으로 그래프 및 링크 보내기

In [36]:
import requests
from bs4 import BeautifulSoup

In [37]:
BASE_URL = 'https://finance.naver.com/marketindex/exchangeDetail.naver?marketindexCd'
parameters = '=FX_EURKRW#'

URL = BASE_URL + parameters
res = requests.get(URL)

res.status_code

200

In [38]:
soup = BeautifulSoup(res.text, 'html.parser')
result = soup.select_one('.flash_area > img') 
graph = result.attrs['src']
#이미지를 크롤링
#속성으로 접근

print(graph)

https://ssl.pstatic.net/imgfinance/chart/marketindex/area/month3/FX_EURKRW.png


In [39]:
import telegram

In [40]:
telegram_config = {}
#config 파일 읽기
with open('./telegram_config', 'r') as f:
    #한 줄씩 읽어오기
    configs = f.readlines()
    #한 줄씩 확인해서
    for config in configs:
        #줄바꿈기호 제거 (\n) 후 =로 문자열 분리
        #key, value로 언패킹 (2개 나올 것이 확실하기 때문에)
        key, value = config.rstrip().split('=')
        #config 딕셔너리에 키-값 추가
        telegram_config[key] = value


print(telegram_config)
token = telegram_config['token']
chatId = telegram_config['chatId']
lastId = telegram_config['lastId']

{'token': '5782216675:AAH9FsDNrxOtRcsj-2ux63aoeQ55tbCwl0Q', 'chatId': '5662031911', 'lastId': '307922339'}


In [41]:
bot = telegram.Bot(token)
updates = bot.get_updates()
updates

In [42]:
chat_id = updates[-1].message.chat.id
chat_id

5662031911

In [43]:
URL = 'https://finance.naver.com/marketindex/exchangeList.naver'
response = requests.get(URL)
soup = BeautifulSoup(response.text, 'html.parser')
results = soup.select('tbody > tr')
country_dic = {}
for result in results:
    title = result.select_one('.tit').text.strip().split(' ')
    country_dic.setdefault(title[0],title[1])
    

In [44]:
country_dic

{'미국': 'USD',
 '유럽연합': 'EUR',
 '일본': 'JPY',
 '중국': 'CNY',
 '홍콩': 'HKD',
 '대만': 'TWD',
 '영국': 'GBP',
 '오만': 'OMR',
 '캐나다': 'CAD',
 '스위스': 'CHF',
 '스웨덴': 'SEK',
 '호주': 'AUD',
 '뉴질랜드': 'NZD',
 '체코': 'CZK',
 '칠레': 'CLP',
 '터키': 'TRY',
 '몽골': 'MNT',
 '이스라엘': 'ILS',
 '덴마크': 'DKK',
 '노르웨이': 'NOK',
 '사우디아라비아': 'SAR',
 '쿠웨이트': 'KWD',
 '바레인': 'BHD',
 '아랍에미리트': 'AED',
 '요르단': 'JOD',
 '이집트': 'EGP',
 '태국': 'THB',
 '싱가포르': 'SGD',
 '말레이시아': 'MYR',
 '인도네시아': 'IDR',
 '카타르': 'QAR',
 '카자흐스탄': 'KZT',
 '브루나이': 'BND',
 '인도': 'INR',
 '파키스탄': 'PKR',
 '방글라데시': 'BDT',
 '필리핀': 'PHP',
 '멕시코': 'MXN',
 '브라질': 'BRL',
 '베트남': 'VND',
 '남아프리카': '공화국',
 '러시아': 'RUB',
 '헝가리': 'HUF',
 '폴란드': 'PLN',
 '스리랑카': 'LKR',
 '알제리': 'DZD',
 '케냐': 'KES',
 '콜롬비아': 'COP',
 '탄자니아': 'TZS',
 '네팔': 'NPR',
 '루마니아': 'RON',
 '리비아': 'LYD',
 '마카오': 'MOP',
 '미얀마': 'MMK',
 '에티오피아': 'ETB',
 '우즈베키스탄': 'UZS',
 '캄보디아': 'KHR',
 '피지': 'FJD'}

In [72]:
# 이미지 파일 함수 만들기
from datetime import datetime
def url(country):
    url1 ='https://ssl.pstatic.net/imgfinance/chart/marketindex/area/month3/FX_'
    url2 = 'KRW.png?'
    today = datetime.today()
    sidcode= int(today.strftime('%Y%m%d'))

    img_url = url1 +country_dic[country] +url2+str(sidcode) ##datetime
    # chat_id = telegram_config['chatId']

    return img_url

In [66]:
# Country_list=[]
# for Country1 in country_dic:
#     Country_list.append(Country1)
# print(Country_list)

In [65]:
# for con in Country_list:
#     img_url=url(con)
#     print(img_url)


In [51]:

# img_url=url('')


In [69]:
img_url

'https://ssl.pstatic.net/imgfinance/chart/marketindex/area/month3/FX_FJDKRW.png?20220919'

In [70]:
# sendphoto 함수
def sendPhoto(country):
    bot.sendPhoto(chat_id=chatId, photo=url(country), caption="현재 환율: " + 환율(country), reply_markup={
        "inline_keyboard": [
                            [
                                {"text": "더 자세한 환율정보를 보시려면 클릭", "url" : 'https://finance.naver.com/marketindex/exchangeDetail.naver?marketindexCd=FX_'+country_dic[country]+'KRW#' }
                            ]
                        ]})

In [71]:
sendPhoto('터키')

# PART 2 축제

# (1) 축제정보 함수 만들기 [def 축제]

In [3]:
from bs4 import BeautifulSoup

In [37]:
def 축제(country):

    headers= {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
    BASE_URL = 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bk5C&qvt=0'
    parameters = '&query=' + country + ' 축제'

    URL = BASE_URL + parameters
    res = requests.get(URL, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    final_result2 = []
    
    #행을 나누기
    festival = soup.select('.scroll tr')
    festival_list = festival[1:]
    # print(festival_list)
    for result in festival_list:
        result_title = result.select('td')[0].text
        result_time = result.select('td')[1].text
        result_place = result.select('td')[2].text

        

        temp = {
            '축제명': result_title,
            '기간(현지기준)': result_time,
            '장소' : result_place
        }

        final_result2.append(temp)
    #country 페이지를 들어가서 페이지 정보를 추출해서 final_result2에 append해줌
    message = ''
    for i in final_result2: #딕셔너리i
        # print(i['축제명'])
        # print(i['기간(현지기준)'])
        # print(i['장소'])
        
        message += i['축제명'] + i['기간(현지기준)'] + i['장소'] + '\n'
    
        #딕셔너리에서 축제명, 기간, 장소로 (key)로 접근해서 값을 가져와라.
    
    return message

' 타이완 자전거 축제   2022.03.01. ~ 12.31.   타이완 주요 도시 \n 타이완등불축제   2022.02.01. ~ 02.28.   타이완 가오슝 \n 타이완 디자인 엑스포   2022.10.07. ~ 10.23.   타이완 신주 \n 르웨탄 국제 불꽃놀이·뮤직·자전거 페스티벌   2022.10.07. ~ 11.19.   타이완 르웨탄 국립 풍경구 \n 난터우 세계 차 박람회   2022.10.08. ~ 10.16.   타이완 난터우 \n 타이완 국제 차 박람회   2022.11.18. ~ 11.21.   타이완 타이페이 난강 전시장 \n 타이완국제열기구축제   2022.07.02. ~ 08.15.   타이완 타이둥 \n 펑후 해상 불꽃축제   2022.04.25. ~ 06.30.   타이완 마공 펑후 파빌리온 레저 공원 \n 타이중 재즈 페스티벌   2022.10.15. ~ 10.23.   타이완 타이중 시민광장 \n 대만 국제 문화 및 창조산업 박람회   2022.08.05. ~ 08.14.   타이완 타이페이 일대 \n 타이완미식전   2022.08.05. ~ 08.08.   타이완 타이페이 \n 타이페이 국제 도서전   2023.01.31. ~ 02.05.   타이완 타이페이세계무역센터 \n 타이페이 국제 아트 페어   2022.10.21. ~ 10.24.   타이완 타이페이 세계 무역 센터 \n 루강 단오절 용선 축제   2022.05.08. ~ 07.17.   타이완 루강·장화해안 일대 \n 보랏빛 나비 계곡 축제   2021.11.06. ~ 02.28.   타이완 가오슝 마오린 국가풍경구 \n 타이중 국제 무용제   2020.10.24. ~ 11.01.   타이완 타이중 \n'

# (2) 나라이름 검색시 해당 나라의 축제 리스트 추출

In [16]:
# festa1 = '미국'

In [17]:
# message = ''
# for i in festa1: #딕셔너리i
#     # print(i['축제명'])
#     # print(i['기간(현지기준)'])
#     # print(i['장소'])
    
#     message += i['축제명'] + i['기간(현지기준)'] + i['장소'] + '\n'
#     #딕셔너리에서 축제명, 기간, 장소로 (key)로 접근해서 값을 가져와라.


TypeError: string indices must be integers

In [81]:
bot.send_message(telegram_config['chatId'], message)

In [85]:
## #cmd_handler_bot.py

In [ ]:
# from telegram.ext import Updater
# from telegram.ext import CommandHandler
 
# BOT_TOKEN='5782216675:AAH9FsDNrxOtRcsj-2ux63aoeQ55tbCwl0Q'
 
# updater = Updater( token=BOT_TOKEN, use_context=True )
# dispatcher = updater.dispatcher
 

# def 나라_환율(update, context):
#     country = 환율(context.args[0])
#     url_cou = url(context.args[0])
#     context.bot.sendPhoto(chat_id=update.effective_chat.id, photo=url_cou, caption="현재 환율: " + country, reply_markup={
#         "inline_keyboard": [
#                             [
#                                 {"text": "더 자세한 환율정보를 보시려면 클릭", "url" : 'https://finance.naver.com/marketindex/exchangeDetail.naver?marketindexCd=FX_'+country_dic[context.args[0]]+'KRW#' }
#                             ]
#                         ]})

 
# def stop(update, context):
#     context.bot.send_message(chat_id=update.effective_chat.id, text="작업을 중단합니다.")
 
# def zigbang(update, context):
#     context.bot.send_message(chat_id=update.effective_chat.id, text="[{}] 주변 매물을 수집합니다.".format( context.args[0] ))
    
 
# start_handler = CommandHandler('country_info', 나라_환율)
# stop_handler = CommandHandler('stop', stop)
# zigbang_handler = CommandHandler('zigbang', zigbang)
 
# dispatcher.add_handler(start_handler)
# dispatcher.add_handler(stop_handler)
# dispatcher.add_handler(zigbang_handler)
 
# updater.start_polling()
# updater.idle()